In [10]:
from wikipedia_parsing import CO2_ratio
from wikipedia_parsing import commuter as commuter_aircraft
from wikipedia_parsing import long_haul as long_haul_aircraft
from wikipedia_parsing import medium_haul as medium_haul_aircraft
from wikipedia_parsing import regional as regional_airfraft
from wikipedia_parsing import short_haul as short_haul_aircraft

import open_flight_exploratory
import pandas as pd

# Aviation and global carbon emissions
Our data was imported and cleaned in 2 helper function that take car of parsing the data and cleaning it. This reduces these long processes to a single import. This will allow use to focus on the analysis of our data in this notebook without being distracted with the fetching and cleaning of the data.

# Carbon emission by class of plane

To start our analysis of what role does the aviation industry play in the global carbon emissions let's look at what type of jet's tends to polute the most. The aircraft data parsed from the wikipedia pages will be used to analyse that type of jets tend to pollute the most.

Note that wikipedia only gave us the fuel burn in kg/km, and L/100km per seat. To get how much one passenger pollutes we need to convert the liters of fuel (jet A-1 keroseen) to: $$\frac{\textit{kg of CO}_2}{100 \text{ km}}$$

According to https://authors.library.caltech.edu/25832/1/galcit_fm99-4.pdf jet fuel is mostly composed of Nonae $C_9H_{20}$. We will assume for the sake of simplicity that jet fuel is composed of a 100% Nonane. We will also make the assumption that aircraft fuel cumbustion is hapening under conditions where the fuel will undergo total combustion. Luckily we remember of chemistry course from high school.

$$C_9H_{20} + 14 O_2 \rightarrow 9CO_2 + 10 H_2O $$


| Element | Molar mass (g) |
|:-------:|:----------:|
|    C    |   12.011   |
|    H    |    1.008   |
|    O    |   15.999   |

Using the mass of each element from the table above we can calculate the molar mass of $C_9H_{20}$ and $CO_2$.

|   Molecule  | Molar mass (g) |
|:-----------:|:----------:|
| $C_9H_{20}$ |   128.259  |
|    $CO_2$   |    43.99   |

From the stoichiometric equation of the complete combustion of one mole of $C_9H_{20}$ we know that nine moles of $CO_2$ will be produced. 

One mole of Nonane will produce:
$$128.259 \, g \,C_9H_{20}= 9* 43.99 \, g \, CO_2$$
$$128.259 \, g \,C_9H_{20}=395.91 \, g \, CO_2$$
$$1 \, g \,C_9H_{20}=3.087 \, g \, CO_2$$

According to wikipedia the density of $C_9H_{20}$ is $0.718 \frac{\text{g}}{\text{ml}}$

In [4]:
regional_airfraft.mean()

Fuel burn kg/km          2.717619
Fuel per seat L/100km    3.351667
CO2 kg/km                8.979013
dtype: float64

In [5]:
commuter_aircraft.mean()

Fuel burn kg/km          2.013333
Fuel per seat L/100km    4.683333
CO2 kg/km                6.652053
dtype: float64

In [6]:
short_haul_aircraft.mean()

Fuel burn kg/km           3.108000
Fuel per seat L/100km     2.687200
CO2 kg/km                10.268832
dtype: float64

In [7]:
medium_haul_aircraft.mean()

Fuel burn kg/km           5.582222
Fuel per seat L/100km     2.743704
CO2 kg/km                18.443662
dtype: float64

In [8]:
long_haul_aircraft.mean()

Fuel burn kg/km           8.138095
Fuel per seat L/100km     2.958095
CO2 kg/km                26.888267
dtype: float64

In [11]:
all_aircraft = pd.concat([commuter_aircraft, regional_airfraft, short_haul_aircraft, medium_haul_aircraft, long_haul_aircraft])

In [12]:
all_aircraft.mean()

Fuel burn kg/km           4.133083
Fuel per seat L/100km     3.221429
CO2 kg/km                13.655705
dtype: float64